<a href="https://colab.research.google.com/github/Shivaranjani1711/Cognifyz-Technology/blob/main/restrarent_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import folium
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
df = pd.read_csv("/content/Dataset .csv")

In [ ]:
df.dropna(subset=['Restaurant Name', 'Cuisines', 'City', 'Average Cost for two', 'Latitude', 'Longitude'], inplace=True)

In [ ]:
df['combined_features'] = df.apply(lambda row: f"{row['Cuisines']} {row['City']} {row['Average Cost for two']}", axis=1)

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
feature_matrix = vectorizer.fit_transform(df['combined_features'])

In [ ]:
def recommend_and_show_map(cuisine, city, budget, top_n=5):
    # Combine user input
    user_input = f"{cuisine} {city} {budget}"
    user_vec = vectorizer.transform([user_input])
    sim_scores = cosine_similarity(user_vec, feature_matrix).flatten()
    top_indices = sim_scores.argsort()[-top_n:][::-1]
    recommended_df = df.iloc[top_indices]
    print("\nTop Recommended Restaurants:\n")
    display(recommended_df[['Restaurant Name', 'Cuisines', 'City', 'Average Cost for two', 'Aggregate rating']])

    map_center = [recommended_df['Latitude'].mean(), recommended_df['Longitude'].mean()]
    restaurant_map = folium.Map(location=map_center, zoom_start=13)

    for _, row in recommended_df.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{row['Restaurant Name']} ({row['Cuisines']}) - ₹{row['Average Cost for two']}",
            tooltip=row['Restaurant Name']
        ).add_to(restaurant_map)

    return restaurant_map



In [ ]:
recommend_and_show_map("North Indian", "Delhi", "600", top_n=6)



Top Recommended Restaurants:



,Restaurant Name,Cuisines,City,Average Cost for two,Aggregate rating
3041,Kake Da Hotel,North Indian,New Delhi,600.0,3.5
3621,Baweja's Haandi,North Indian,New Delhi,600.0,0.0
4110,Kohli Dhaba,North Indian,New Delhi,600.0,3.4
3293,Kukkad Nukkad,North Indian,New Delhi,600.0,3.0
3877,Singh Dhaba,North Indian,New Delhi,600.0,3.0
3367,Deep Chicken Point,North Indian,New Delhi,600.0,3.0
